# Intermediate Python

---

## Command line Arguments

# argparse

* * *

  

bogotobogo.com

* * *

## What is argparse?

What is argparse?

"The argparse module makes it easy to write user-friendly command-line interfaces. The program defines what arguments it requires, and argparse will figure out how to parse those out of **sys.argv**. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments." - from [argparse - Parser for command-line options, arguments and sub-commands](https://docs.python.org/2/library/argparse.html)

The following description is from [The argparse module is now part of the Python standard library!](https://code.google.com/p/argparse/)

The argparse module provides an easy, declarative interface for creating command line tools, which knows how to:

1. parse the arguments and flags from sys.argv
2. convert arg strings into objects for your program
3. format and print informative help messages

and much more...

The argparse module improves on the standard library optparse module in a number of ways including:

1. handling positional arguments
2. supporting sub-commands
3. allowing alternative option prefixes like + and /
4. handling zero-or-more and one-or-more style arguments
5. producing more informative usage messages
6. providing a much simpler interface for custom types and actions
  
  

A simple sample

Let's look at our first sample of using argparse:

# arg.py

In [ ]:
%%writefile arg.py
import argparse
import sys

def check_arg(args=None):
    parser = argparse.ArgumentParser(description='Script to learn basic argparse')
    parser.add_argument('-H', '--host',
                        help='host ip',
                        required='True',
                        default='localhost')
    parser.add_argument('-p', '--port',
                        help='port of the web server',
                        default='8080')
    parser.add_argument('-u', '--user',
                        help='user name',
                        default='root')

    results = parser.parse_args(args)
    return (results.host,
            results.port,
            results.user)

if __name__ == '__main__':
    h, p, u = check_arg(sys.argv[1:])
    print('h =',h)
    print('p =',p)
    print('u =',u)

If we run it:

In [ ]:
!python arg.py -H 192.17.23.5

Note that the 'host' arg is set as 'required'. So, if we run the code without feeding host ip, we'll get an error like this:

In [ ]:
!python arg.py

Help option

Also, we need to look at how the help works:

In [ ]:
!python arg.py -h

Script to learn basic argparse

    optional arguments:
        -h, --help            show this help message and exit
        -H HOST, --host HOST  host ip
        -p PORT, --port PORT  port of the web server
        -u USER, --user USER  user name

Notice that we used **-H** for host-ip mandatory option instead of lower case 'h' because it is reserved for 'help.

  
  
  

Integer input

Another sample code:

# arg.py

In [ ]:
%%writefile arg2.py
import argparse
import sys

def int_args(args=None):
    parser = argparse.ArgumentParser(description='Processing integers.')
    parser.add_argument('integers',
                        metavar='N',
                        type=int,
                        nargs='+',
                        help='integer args')
    return parser.parse_args()

if __name__ == '__main__':
    print(int_args(sys.argv[1:]))

Just to see how it works, let's request 'help':

In [ ]:
!python arg2.py -h

### Processing integers.

positional arguments:
  N           integer args

optional arguments:
  -h, --help  show this help message and exit

We need to check the **add_argument()** method in **ArgumentParser.add_argument()**:

ArgumentParser.add_argument(name or flags...[, action][, nargs][, const][, default][, type][, choices][, required][, help][, metavar][, dest])

1. **name or flags** - Either a name or a list of option strings, e.g. foo or -f, --foo.
2. **action** - The basic type of action to be taken when this argument is encountered at the command line.
3. **nargs** - The number of command-line arguments that should be consumed.
4. **const** - A constant value required by some action and nargs selections.
5. **default** - The value produced if the argument is absent from the command line.
6. **type** - The type to which the command-line argument should be converted.
7. **choices** - A container of the allowable values for the argument.
8. **required** - Whether or not the command-line option may be omitted (optionals only).
9. **help** - A brief description of what the argument does.
10. **metavar** - A name for the argument in usage messages.
11. **dest** - The name of the attribute to be added to the object returned by parse_args().
  

Now, it's time to run the code:

In [ ]:
!python arg2.py 1 2 3 4 5

The '+' in nargs='+', just like '*', makes all command-line args present to be gathered into a list. 

If we provide a wrong type arg such as a float type, we'll get an error:

In [ ]:
!python arg2.py 1.9999 2 3 4 5

---

## A full example

In [ ]:
%%writefile args3.py
import json
import requests
import time
import calendar
import argparse
import sys
import operator

DAYS = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
WEEK_IN_SECOND = 604800

def count_commits(repo, weeks, sort):

    # Get commit activity via Github REST API v3 with Python's requests module
    r = requests.get('https://api.github.com/repos/%s/stats/commit_activity' % repo)

    repos = json.loads(r.content)
    # sample : [{'days': [0, 0, 0, 0, 0, 0, 0], 'total': 0, 'week': 1511053200}, ... ]

    # calculate time cut in weeks
    current_epoctime = calendar.timegm(time.gmtime())
    week_cut = current_epoctime-(weeks)*WEEK_IN_SECOND

    # initialize commits for the days of a week
    commits = [0]*7
    # loop through commits week by week. Cut by input 'weeks'
    for r in repos:
       if r['week'] >= week_cut:
           for i,d in enumerate(r['days']):
               commits[i] += d

    # average commits per week
    commits = [c/weeks for c in commits]

    # construct dictionary from the two list : zip(DAYS, commits)
    # then, sort it (default: descending)
    days_commits = dict(zip(DAYS, commits))
    days_commits = sorted(days_commits.items(), key = operator.itemgetter(1), reverse = (sort == 'dsc'))
    # sample :  days_commits =  [('Wednesday', 75.95), ('Thursday', 73.7), ... ]

    print('\n--- Commits (average) ---')
    for item in days_commits:
        print('%s %.1f' %(item[0],item[1]))

    print('\n--- The most commits ---')
    if sort == 'dsc':
        index = 0  # top
    else:
        index = 6  # bottom
    print('%s %.1f' %(days_commits[index][0],days_commits[index][1]))

# setup args including default values and input error handling
def check_arg(args=None):
    parser = argparse.ArgumentParser(description='Github API - stats of commits')
    #parser.add_argument('repo', metavar='repository name', type=str, help='user/repo')
    parser.add_argument('-r', nargs='?', default='kubernetes/kubernetes')
    parser.add_argument('-w', nargs='?', default='52')
    parser.add_argument('-s', nargs='?', default='dsc')
    results = parser.parse_args(args)
    return (results.r, results.w, results.s)

# MAIN
if __name__ == '__main__':

    '''
    Count Github commits via 'stats/commit_activity' - using REST API v3
    Usage :  python ask_ki.py -r kubernetes/kubernetes -w=36 -s=dsc
    (note) All args are optional: '-r', '-w', and '-s'
    '''

    r, w, s = check_arg(sys.argv[1:])
    print('Inputs: repo=%s weeks=%s sort=%s' %(r,w,s))

    # call github api
    count_commits(repo=r, weeks=int(w), sort=s)

In [ ]:
%run args3.py

---

Wrap

Now you’ve seen the basics of `argparse`. You’ve seen how to get positional and optional arguments into your programs from the command line. You’ve also seen how to set default optional arguments. If you want to go deeper, check out the [official docs](https://docs.python.org/3/library/argparse.html).

 I should mention that there are a number of packages available to add command line arguments to your program. Some of the most popular are:

*   [click](https://click.palletsprojects.com/en/7.x/)
*   [fire](https://github.com/google/python-fire)
*   [docopt](https://github.com/docopt/docopt)


Python script template with argparse in it: https://gist.github.com/burkesquires/2bab01406597312a2ef0cc74128df89f

You can also check out [python fire](https://github.com/google/python-fire) from Google that generates CLI